# Test-Time Adaptation

**Bối cảnh:** Khi mô hình được huấn luyện ở *source domain* (train/val) nhưng triển khai ở *target domain* (ảnh bị nhiễu, mờ, thay đổi độ sáng...), độ chính xác suy giảm. **Test-Time Adaptation (TTA)** cập nhật **thống kê chuẩn hoá** hoặc **một phần tham số** ngay tại test-time **không dùng nhãn**.

Trong notebook này, ta cài đặt **2 thuật toán TTA khác nhau** (không chỉ tune tham số):

1. **BN Update (Recalibration)** — chỉ **cập nhật running stats của BatchNorm** bằng dữ liệu target *không nhãn*, **không backprop**.
   - Biến thể:
     - **BN-Update (m=0.1)** — momentum mặc định.
     - **BN-Update (m=0.01)** — momentum thấp (chậm mà chắc), giảm nhiễu.
2. **TENT (Test-time Entropy Minimization)** — **tối ưu entropy** của phân phối dự đoán trên dữ liệu target để mô hình trở nên **tự tin** hơn.
   - Biến thể:
     - **TENT-Adam** — tối ưu **chỉ** các tham số **affine của BN (gamma/beta)** bằng Adam.
     - **TENT-SGD-noRS** — tối ưu BN affine bằng SGD **đồng thời khoá running stats** (không cập nhật thống kê) để giảm dao động.

**Cách chọn “best” mà không dùng nhãn:** chia target (test bị nhiễu) thành **adapt set** (để cập nhật) và **target-val set** (để **early stopping** bằng **entropy trung bình** – càng thấp càng tốt). Đánh giá cuối trên phần **target-test** còn lại.

**Checkpoint đầu ra** (dạng ví dụ, phụ thuộc tên mô hình gốc `${base}`):
- `checkpoints/tta_bnupdate_m010_${base}.pth`, `checkpoints/tta_bnupdate_m001_${base}.pth`
- `checkpoints/tta_tent_adam_${base}.pth`, `checkpoints/tta_tent_sgd_nors_${base}.pth`



## (A) BN Update (Recalibration)

**Ý tưởng:** tại test-time, đưa mô hình sang `train()` **nhưng không bật gradient**, chạy qua **target adapt set** để **cập nhật running mean/var của BN**.  
- **Không cập nhật weight** nào khác, **chỉ** running stats.  
- **Biến thể**:
  - **m=0.1** — cập nhật nhanh (thích nghi mạnh hơn, có thể nhiễu).  
  - **m=0.01** — cập nhật chậm (ổn định hơn).  
**Early stopping:** chọn bản tốt nhất theo **entropy** trên **target-val** (không dùng nhãn).


## (B) TENT — Test-Time Entropy Minimization

**Ý tưởng:** tại test-time, tối ưu **entropy** của phân phối dự đoán \\(p(y|x)\\) để mô hình **tự tin** hơn trên target *không nhãn*.  
- **Thực hành chuẩn:** **chỉ tối ưu affine của BN** (gamma/beta), giữ nguyên các tham số khác.  
- **BatchNorm mode:** dùng **batch statistics** (đặt `train()`), có thể **tắt cập nhật running stats** để giảm dao động.  
- **Biến thể:**
  - **TENT-Adam:** Adam lr=1e-3, **có** cập nhật running stats.
  - **TENT-SGD-noRS:** SGD lr=1e-3, **không** cập nhật running stats (đặt `bn.momentum=0.0` trong quá trình adapt).

**Early stopping (không nhãn):** theo **entropy** trên **target-val** (nhỏ hơn là tốt).
